# Notebook 12: Link Spam and Topic-Specific PageRank
***

In this notebook we'll examine the how topic-specific PageRank can be leveraged to weed out spam pages and diagnose spam.

We'll need numpy for this notebook, so let's load it.

In [2]:
import numpy as np

<br>

### Exercise 0: Following along with lecture is cool!

With the random teleports (to solve the spider traps and dead ends problems), our transition matrix $A$ is constructed of:
$$A_{ij} = \beta M_{ij} + (1-\beta)/N$$
where $M$ is the original transition matrix, $\beta$ is a probability (typically between 0.8 and 0.9) of taking an actual out-link (as opposed to a teleport), and $N$ is the number of pages. These elements $A_{ij}$ are the probability of transitioning from page $j$ to page $i$ (possibly a bit sideways from what you might intuitively think, but it makes the power iteration work out).

With ***topic-specific*** PageRank, however, we restrict the teleport set $S$ to only a subset of the pages (on the topic of interest). Our transition matrix becomes:
$$A_{ij} = \begin{cases} \beta M_{ij} + (1-\beta)/|S| & \text{if } i \in S \\ \beta M_{ij} & \text{otherwise} \end{cases}$$

For the example of slide 10, we have the following graph:

<img width=200px src="http://www.cs.colorado.edu/~anwo7157/home/resources/topicspecificpagerank1.png">

We saw that the transition matrix $M$ for this graph is:

In [3]:
M = np.array([[1/2, 1/3, 0, 1/2],
              [1/2, 1/3, 0, 0  ],
              [0  , 1/3, 0, 1/2],
              [0  , 0  , 1, 0  ]])

Using $\beta = 0.8$ and the teleport set $S = \{1,2\}$, we can make a slight modification of our previous code to make it topic-specific. In particular, in the matrix that is associated with the teleports (the $1-\beta$ term in $A$), we want 1s in the rows corresponding to the teleport set's pages, and 0s in the other rows. In our original code, we have 1s everywhere. We also need to account for division by the size of the teleport set, $|S|$, instead of the full set of pages.

In [ ]:
# OLD code:
# TODO -- can you cleverly modify the line with `Nmat`
#         to only have 1s for the in-topic nodes? You can
#         use more than 1 line if that makes life easier.
n = 4
beta = 0.8
Nmat = np.ones((n,n))/n # TODO -- modify this line for topic-specific PageRank
A = beta*M + (1-beta)*Nmat

In [8]:
# SOLUTION:

n = 4
beta = 0.8
Nmat = np.zeros((n,n))
S = [1, 2]
for i in S:
    Nmat[i-1,:] = np.ones(n)/len(S)
A = beta*M + (1-beta)*Nmat

To iterate, we need to initialize our PageRank vector, which we can do using an even distribution of rank:

In [21]:
r_old = np.repeat(1/n, n)
r_new = np.repeat(1/n, n)

And we can perform 10 iterations of power iteration, and make sure we agree with the lecture results:

In [ ]:
print("    1      2      3      4")
print("{:2.0f}  {:0.3f}  {:0.3f}  {:0.3f}  {:0.3f}".format(0, r_old[0],r_old[1],r_old[2],r_old[3]))
for t in range(10):
    r_new = 0 # TODO -- compute new rank vector from the old one
    r_old = 0 # TODO -- step the old one forward for the next iteration
    # Renormalize r_new to make sure it sums to 1:
    #   1) obtain the sum of the elements in r_new
    mag = 0 #   TODO
    #   2) divide each element of r_new by the sum of all of them
    r_new = 0 # TODO
    print("{:2.0f}  {:0.3f}  {:0.3f}  {:0.3f}  {:0.3f}".format(t+1, r_new[0],r_new[1],r_new[2],r_new[3]))

<br>

### Exercise 1: Spam farming

Recall the original graph we looked at when learning power iteration (notebook 10).

<img width=250px src="http://www.cs.colorado.edu/~anwo7157/home/resources/pagerank1.png">

Let's obtain the PageRanks for each of the 5 pages using a teleport probability of 0.15 ($\beta = 0.85$). We'll use the compact representation of the transition matrix $M$, since this will lend itself well to expanding our web graph as the devious owner of Page 4 starts up a **spam farm**!

In [ ]:
# TODO: fill in the compact representation for M
M_compact = [[Page 1 out-degree, [Page 1 destinations]],
             [Page 2 out-degree, [Page 2 destinations]],
             [Page 3 out-degree, [Page 2 destinations]],
             [Page 4 out-degree, [Page 3 destinations]],
             [Page 5 out-degree, [Page 5 destinations]]]

In [68]:
# SOLUTION:
N = 5
M = np.zeros((N,N))
M[:,0] = [0, 1, 0, 0, 0]
M[:,1] = [0, 0, 0, 0, 1]
M[:,2] = [1/2, 0, 0, 1/2, 0]
M[:,3] = [1/3, 0, 1/3, 0, 1/3]
M[:,4] = [0, 1/2, 1/2, 0, 0]
print(M)

M_compact = {}
for i in range(N):
    M_compact[i+1] = [idx+1 for idx, x in enumerate(M[:,i]) if x > 0]
print(M_compact)

Stealing liberally from our own previous code, we can initialize and run 50 iterations of power iteration as follows:

In [42]:
# initialize for power iteration
n = len(M_compact)
beta = 0.85
r_old = np.repeat(1/n, n)

for t in range(50):
    # initialize with the "taxed" importance
    r_new = np.repeat((1-beta)/n, n)
    # distribute importance payments between the nodes
    for page in range(n):
        for dest in M_compact[page][1]:
            idx = dest-1 # accounting for Python's 0-based indexing
            r_new[idx] += beta*r_old[page]/M_compact[page][0]
    # normalize to ensure r_new sums to 1
    r_new = r_new/np.sum(r_new)
    r_old = r_new

print(np.round(r_new, 4))

[0.1372 0.389  0.0466 0.0382 0.389 ]


**Consider:** What line would we need to change if we wanted to use *topic-specific* PageRank? And why?

The devious owner of Page 4, Franklin, is sad that his page has such low rank. In an effort to boost it, he decides to create a **spam farm**. He doesn't want to go all in at once, so first Franklin creates a single spam page (Page 6), with a single out-link to Page 4, and a single in-link from Page 4.

Append the link information for Page 6 to `M_compact`. Don't forget to update the link information for Page 4, as well as Page 6.

And recompute the PageRank for all of the pages. Don't forget to update the number of pages!

In [95]:
# initialize for power iteration
n = len(M_compact)
beta = 0.85
r_old = np.repeat(1/n, n)

for t in range(50):
    # initialize with the "taxed" importance
    r_new = np.repeat((1-beta)/n, n)
    # distribute importance payments between the nodes
    for page in range(n):
        for dest in M_compact[page][1]:
            idx = dest-1 # accounting for Python's 0-based indexing
            r_new[idx] += beta*r_old[page]/M_compact[page][0]
    # normalize to ensure r_new sums to 1
    r_new = r_new/np.sum(r_new)
    r_old = r_new

print(np.round(r_new, 4))

[0.0637 0.1395 0.0306 0.6439 0.0997 0.0227]


We're going to need the full PageRank for all of these pages later, so let's hold onto it.

In [96]:
pagerank = r_new

By how much did the rank of Page 4 increase as a result of this spam farm page? The following equation was derived in class as the theoretical increase in PageRank of a target page as a result of $M$ spam pages feeding into it:

$$y = x + \beta M \left[\dfrac{\beta y}{M} + \dfrac{1-\beta}{N}\right] + \dfrac{1-\beta}{N}$$

where $y$ is the PageRank of the target page and $x$ is the rank contributed by good, upstanding pages. Is there some part of this equation that needs to be modified to fit our graph with the added spam page? Why or why not?

**Solution:** 


You should have found that the increase in PageRank for Franklin was large. Why did adding a single farm page yield such a huge benefit?

<br>

### Exercise 2: Franklin's Treachery, and TrustRank, the hero we need!

<img width=280px src="http://www.cs.colorado.edu/~anwo7157/home/resources/linkspam1.png">

Now that our world is crashing down around us because of the treachery of Franklin, we need to figure out how to combat his admittedly sad little spam farm. Use the teleport set $S = \{1,2,3,4,5\}$ and $\beta = 0.85$ to perform 50 iterations of power iteration and estimate *TrustRank* with respect to the trusted set $S$ (the original pages). Note that you will likely need to change the "initialize with the taxed importance" step in the code we used previously, to account for teleports only to the trusted set.

In [1]:
# SOLUTION:

The rank of Page 4 hasn't really decreased by very much, which might be discouraging. After all, if we haven't knocked Franklin down a peg, what is all this really about?

But wait! We can compute the **spam mass** of each page as:
$$m = \dfrac{r - t}{r}$$
where $r$ is the PageRank (including the spam farm page) and $t$ is the TrustRank (with respect to only the trusted set $S$).

So our spam masses for each page are:

In [ ]:
spammass = (pagerank - r_new)/pagerank
print(spammass)

What do you notice about Franklin and his farm?